In [4]:
import pandas as pd
import os
import streamlit as st
from dotenv import load_dotenv
# from PyPDF2 import PdfReader, PdfFileReader
# import PyPDF2
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
# from htmlTemplates import css, bot_template, user_template
from langchain.llms import HuggingFaceHub
import tiktoken
import urllib3


In [5]:
load_dotenv() # this automatically load my OPENAI_API_KEY from the .env file without having to assign it to any variable or parameter

True

# Interact with GPT using a simple prompt

In [12]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

prompt_template = """You are an expert about policies of ITL Corporation, I will ask you a question, and then provide you some chunks of text contain relevant information about the question. 
Try to answer the question based on the text provided in Vietnamese.
Questions: 
{user_question}  

Relevant Information:
{relevant_info}
"""

llm = OpenAI(temperature=0)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)


In [18]:
user_question = 'nguyễn quốc thúc là ai?'
relevant_info = """
nội quy lao động

	01/07/2023

	mã tài liệu: hr.03.v3.2023

	thẩm định

	phê duyệt

	nguyễn quốc thúc (simon)
	phó tgđ – vp nhân sự	

	nguyễn thị kim oanh (kimberly)
	vp khối phát triển bền vững


	trần tuấn anh (ben)
	tổng giám đốc

	

	soạn thảo & đồng thuận

	võ thị ngọc mai (jina)
	trưởng phòng tiền lương và phúc lợi

	nguyễn thị phương thảo (oliver)
	trưởng phòng nhân sự cấp cao


	lê thị hương (minty)
	phó phòng pháp lý
	

	nguyễn thị minh hải (hellen)
	trưởng phòng tuân thủ


	lịch sử sửa đổi tài liệu

	mã tài liệu

	ngày hiệu lực

	nội dung thay đổi

	trang

	-

	21/02/2019

	cập nhật tài liệu

	-

	hr.03.v2.2021

	01/04/2021

	cập nhật tài liệu theo quy định của luật lao động 2019

	toàn bộ

	hr.03.v3.2023

	01/07/2023

	 cập nhật tài liệu

	toàn bộ

"""

response = llm_chain(
    {
        'user_question': user_question,
        'relevant_info': relevant_info
    }
)
print(response['text'])

Nguyễn Quốc Thúc là Phó Tổng Giám Đốc - VP Nhân Sự của Công ty ITL.


In [19]:
response = llm_chain(
    {
        'user_question': "nội dung này được soạn thảo bởi ai?",
        'relevant_info': relevant_info
    }
)
print(response['text'])

Nội dung này được soạn thảo bởi Võ Thị Ngọc Mai (Jina), Nguyễn Thị Phương Thảo (Oliver), Lê Thị Hương (Minty), Nguyễn Thị Minh Hải (Hellen).


In [20]:
llm('how many provinces vietname has?')

'\n\nVietnam has 63 provinces.'

In [24]:
# print this out so you can see how does promt_template work
print(prompt_template.format(user_question="tai lieu nay duoc soan thao khi nao?", relevant_info=relevant_info)) 

You are an expert about policies of ITL Corporation, I will ask you a question, and then provide you some chunks of text contain relevant information about the question. 
Try to answer the question based on the text provided in Vietnamese.
Questions: 
tai lieu nay duoc soan thao khi nao?  

Relevant Information:

nội quy lao động

	01/07/2023

	mã tài liệu: hr.03.v3.2023

	thẩm định

	phê duyệt

	nguyễn quốc thúc (simon)
	phó tgđ – vp nhân sự	

	nguyễn thị kim oanh (kimberly)
	vp khối phát triển bền vững


	trần tuấn anh (ben)
	tổng giám đốc

	

	soạn thảo & đồng thuận

	võ thị ngọc mai (jina)
	trưởng phòng tiền lương và phúc lợi

	nguyễn thị phương thảo (oliver)
	trưởng phòng nhân sự cấp cao


	lê thị hương (minty)
	phó phòng pháp lý
	

	nguyễn thị minh hải (hellen)
	trưởng phòng tuân thủ


	lịch sử sửa đổi tài liệu

	mã tài liệu

	ngày hiệu lực

	nội dung thay đổi

	trang

	-

	21/02/2019

	cập nhật tài liệu

	-

	hr.03.v2.2021

	01/04/2021

	cập nhật tài liệu theo quy định của luật 

# Memory: add memory to the bot

In [25]:
from langchain import ConversationChain

In [26]:
ConversationChain?

Init signature:
ConversationChain(
    *,
    memory: langchain.schema.memory.BaseMemory = None,
    callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain.callbacks.base.BaseCallbackManager] = None,
    verbose: bool = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    prompt: langchain.schema.prompt_template.BasePromptTemplate = PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'),
    llm: langchain.schema.language_model.BaseLanguageModel,
    output_key: str = 'response',
    output_parser: langchain.schema.output_parser.BaseLL

In [28]:
conversation = ConversationChain(llm=llm, verbose=True)

In [29]:
conversation.predict(input="Hi there")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there
AI:

> Finished chain.


' Hi there! How can I help you?'

In [31]:
conversation.predict(input="I am just trying to learn langchain, which is a new library that support create NLP applications easily. Do you know about langchain?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hi there! How can I help you?
Human: I am just trying to learn langchain, which is a new library that support create NLP applications easily. Do you know about langchain?
AI:

> Finished chain.


" I'm sorry, I don't know about langchain. Could you tell me more about it?"

In [32]:
conversation.predict(input="yeah, as I predicted, you do not know about it, because you have been trained only on data from 2021 backward")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hi there! How can I help you?
Human: I am just trying to learn langchain, which is a new library that support create NLP applications easily. Do you know about langchain?
AI:  I'm sorry, I don't know about langchain. Could you tell me more about it?
Human: yeah, as I predicted, you do not know about it, because you have been trained only on data from 2021 backward
AI:

> Finished chain.


' I see. It sounds like langchain is a relatively new library. Could you tell me more about what it does and how it can help create NLP applications?'

In [33]:
conversation.predict(input='unfortunately, I still do not have enough knowledge & understanding about the library to clearly explain to you in detail about it. anyway, just want to check, do you remember what am I learning?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there
AI:  Hi there! How can I help you?
Human: I am just trying to learn langchain, which is a new library that support create NLP applications easily. Do you know about langchain?
AI:  I'm sorry, I don't know about langchain. Could you tell me more about it?
Human: yeah, as I predicted, you do not know about it, because you have been trained only on data from 2021 backward
AI:  I see. It sounds like langchain is a relatively new library. Could you tell me more about what it does and how it can help create NLP applications?
Human: unfortunately, I still do not have enough knowledge & understanding about the library to clearly explain to you in detail about i

' Yes, you mentioned that you are trying to learn langchain, which is a new library that supports creating NLP applications easily. Is that correct?'

Good, now, my chatbot can remember things 

# Embedding

In [35]:
from langchain.document_loaders import TextLoader, Docx2txtLoader

In [81]:
loader=Docx2txtLoader("../data/raw/6. HR.03.V3.2023. Nội quy Lao động_Review by Labor Department - Final.DOCX")
nqld=loader.load()#[0].page_content

In [44]:
print(nqld)

NỘI QUY

	LAO ĐỘNG

	01/07/2023

	Mã tài liệu: HR.03.V3.2023

	

Hội đồng soạn thảo 

	

	THẨM ĐỊNH

	PHÊ DUYỆT

	

	

	

	

	

	

	

	Nguyễn Quốc Thúc (Simon)

	Phó TGĐ – VP Nhân sự

	

	

	

	

	

	

	

	Nguyễn Thị Kim Oanh (Kimberly)

	VP Khối Phát Triển Bền vững

	

	

	

	

	

	

	

	Trần Tuấn Anh (Ben)

	Tổng Giám đốc

	

	SOẠN THẢO

	ĐỒNG THUẬN

	

	

	

	

	

	

	

	Võ Thị Ngọc Mai (Jina)

	Trưởng phòng Tiền lương và Phúc lợi

	

	

	

	

	

	

	

	Nguyễn Thị Phương Thảo (Oliver)

	Trưởng phòng Nhân sự cấp cao

	

	

	

	

	

	

	

	

	Lê Thị Hương (Minty)

	Phó phòng Pháp lý

	

	

	

	

	

	

	

	Nguyễn Thị Minh Hải (Hellen)

	Trưởng phòng Tuân Thủ

	

LỊCH SỬ SỬA ĐỔI TÀI LIỆU

	

	Mã tài liệu

	Ngày hiệu lực

	Nội dung thay đổi

	Trang

	-

	21/02/2019

	Cập nhật tài liệu

	-

	HR.03.V2.2021

	01/04/2021

	Cập nhật tài liệu theo quy định của Luật lao động 2019

	Toàn bộ

	HR.03.V3.2023

	01/07/2023

	 Cập nhật tài liệu

	Toàn bộ

	

	


	

	

CÔNG TY CỔ PHẦN GIAO NHẬN VÀ VẬN

In [48]:
text_splitter=CharacterTextSplitter(
        separator=".",
        chunk_size=500,
        chunk_overlap=100,
        length_function=len
    )

In [83]:
doc_chunks=text_splitter.split_documents(nqld)

Created a chunk of size 746, which is longer than the specified 500
Created a chunk of size 1162, which is longer than the specified 500
Created a chunk of size 1949, which is longer than the specified 500
Created a chunk of size 1468, which is longer than the specified 500
Created a chunk of size 1514, which is longer than the specified 500
Created a chunk of size 1382, which is longer than the specified 500
Created a chunk of size 655, which is longer than the specified 500
Created a chunk of size 1770, which is longer than the specified 500
Created a chunk of size 1726, which is longer than the specified 500
Created a chunk of size 1081, which is longer than the specified 500
Created a chunk of size 2193, which is longer than the specified 500
Created a chunk of size 573, which is longer than the specified 500
Created a chunk of size 2982, which is longer than the specified 500
Created a chunk of size 904, which is longer than the specified 500
Created a chunk of size 646, which is 

In [84]:
doc_chunks

[Document(page_content='NỘI QUY\n\n\tLAO ĐỘNG\n\n\t01/07/2023\n\n\tMã tài liệu: HR.03.V3', metadata={'source': '../data/raw/6. HR.03.V3.2023. Nội quy Lao động_Review by Labor Department - Final.DOCX'}),
 Document(page_content='2023\n\n\t\n\nHội đồng soạn thảo \n\n\t\n\n\tTHẨM ĐỊNH\n\n\tPHÊ DUYỆT\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tNguyễn Quốc Thúc (Simon)\n\n\tPhó TGĐ – VP Nhân sự\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tNguyễn Thị Kim Oanh (Kimberly)\n\n\tVP Khối Phát Triển Bền vững\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tTrần Tuấn Anh (Ben)\n\n\tTổng Giám đốc\n\n\t\n\n\tSOẠN THẢO\n\n\tĐỒNG THUẬN\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tVõ Thị Ngọc Mai (Jina)\n\n\tTrưởng phòng Tiền lương và Phúc lợi\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tNguyễn Thị Phương Thảo (Oliver)\n\n\tTrưởng phòng Nhân sự cấp cao\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tLê Thị Hương (Minty)\n\n\tPhó phòng Pháp lý\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\

In [61]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [85]:
text_splitter2 = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
    chunk_size=500,
    chunk_overlap=50,
    length_function=len
)

In [87]:
text_chunks=text_splitter2.split_documents(nqld)

In [89]:
for i in text_chunks:
    # print(len(i))
    print(i)

page_content='NỘI QUY\n\n\tLAO ĐỘNG\n\n\t01/07/2023\n\n\tMã tài liệu: HR.03.V3.2023\n\n\t\n\nHội đồng soạn thảo \n\n\t\n\n\tTHẨM ĐỊNH\n\n\tPHÊ DUYỆT\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tNguyễn Quốc Thúc (Simon)\n\n\tPhó TGĐ – VP Nhân sự\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tNguyễn Thị Kim Oanh (Kimberly)\n\n\tVP Khối Phát Triển Bền vững\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tTrần Tuấn Anh (Ben)\n\n\tTổng Giám đốc\n\n\t\n\n\tSOẠN THẢO\n\n\tĐỒNG THUẬN\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tVõ Thị Ngọc Mai (Jina)\n\n\tTrưởng phòng Tiền lương và Phúc lợi\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tNguyễn Thị Phương Thảo (Oliver)' metadata={'source': '../data/raw/6. HR.03.V3.2023. Nội quy Lao động_Review by Labor Department - Final.DOCX'}
page_content='Nguyễn Thị Phương Thảo (Oliver)\n\n\tTrưởng phòng Nhân sự cấp cao\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tLê Thị Hương (Minty)\n\n\tPhó phòng Pháp lý\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n

In [90]:
openai_embedder = OpenAIEmbeddings()
db = FAISS.from_documents(text_chunks, openai_embedder)

In [94]:
query="Trường hợp điều chuyển trên 60 ngày làm việc"
db.similarity_search_with_score(query, k=3)

[(Document(page_content='Trường hợp điều chuyển trên 60 ngày làm việc sẽ thỏa thuận với Người Lao Động bằng văn bản.\n\n\t\n\nĐiều 40: Tạm Hoãn Hợp Đồng Lao Động\n\n\t\n\n\tNgoài các trường hợp tạm hoãn hợp đồng lao động theo luật định, Công ty xem xét thỏa thuận tạm hoãn hợp đồng lao động trong các trường hợp sau:\n\n\tKhi gặp khó khăn đột xuất do thiên tai, dịch bệnh, hỏa hoạn.\n\n\tDo nhu cầu sản xuất kinh doanh: tái cấu trúc nội bộ, thay đổi chiến lược kinh doanh, thu hẹp hoạt động sản xuất kinh doanh.', metadata={'source': '../data/raw/6. HR.03.V3.2023. Nội quy Lao động_Review by Labor Department - Final.DOCX'}),
  0.18671358),
 (Document(page_content='Người Lao Động chuyển sang làm công việc khác so với hợp đồng lao động được trả lương theo công việc mới. Nếu tiền lương của công việc mới thấp hơn tiền lương của công việc cũ thì được giữ nguyên tiền lương của công việc cũ trong thời hạn 30 ngày làm việc. Tiền lương theo công việc mới ít nhất phải bằng 85% tiền lương của công v

In [96]:
db.save_local(folder_path="../data/interim/")

In [97]:
text_chunks

[Document(page_content='NỘI QUY\n\n\tLAO ĐỘNG\n\n\t01/07/2023\n\n\tMã tài liệu: HR.03.V3.2023\n\n\t\n\nHội đồng soạn thảo \n\n\t\n\n\tTHẨM ĐỊNH\n\n\tPHÊ DUYỆT\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tNguyễn Quốc Thúc (Simon)\n\n\tPhó TGĐ – VP Nhân sự\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tNguyễn Thị Kim Oanh (Kimberly)\n\n\tVP Khối Phát Triển Bền vững\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tTrần Tuấn Anh (Ben)\n\n\tTổng Giám đốc\n\n\t\n\n\tSOẠN THẢO\n\n\tĐỒNG THUẬN\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tVõ Thị Ngọc Mai (Jina)\n\n\tTrưởng phòng Tiền lương và Phúc lợi\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tNguyễn Thị Phương Thảo (Oliver)', metadata={'source': '../data/raw/6. HR.03.V3.2023. Nội quy Lao động_Review by Labor Department - Final.DOCX'}),
 Document(page_content='Nguyễn Thị Phương Thảo (Oliver)\n\n\tTrưởng phòng Nhân sự cấp cao\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\t\n\n\tLê Thị Hương (Minty)\n\n\tPhó phòng Pháp lý\n\n\t\n\n\t\n\